In [ ]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import sklearn.metrics as metrics 

###### 前面的四個模型分別對trainset與testset做預測得出結果後，作為此模型的training input 與 testing input

df = pd.read_csv('6CDRclass_predict_train_class.csv',index_col = [0])
df_val = pd.read_csv('6CDRclass_predict_val_class.csv',index_col = [0])
######
df_class = pd.read_csv('CN_MCI_other_predict_train_class.csv')
df_class_val = pd.read_csv('CN_MCI_other_predict_val_class.csv')
######
df_1212class = pd.read_csv('1212class_predict_train.csv')
df_1212class_val = pd.read_csv('1212class_prediction_val.csv')
######
df_3111 = pd.read_csv('3111class_prediction_train.csv')
df_3111_val = pd.read_csv('3111class_prediction_val.csv')
######

df['PTGENDER'] = df['PTGENDER'].replace("Female", 0).replace("Male", 1)
df_val['PTGENDER'] = df_val['PTGENDER'].replace("Female", 0).replace("Male", 1)
df_class = df_class[['DX_pred_1ST','DX_pred_2ND','DX_pred_3RD']]
df_class_val = df_class_val[['DX_pred_1ST','DX_pred_2ND','DX_pred_3RD']]
df_1212class = df_1212class[['DX4_pred_1ST','DX4_pred_2ND','DX4_pred_3RD','DX4_pred_4TH']]
df_1212class_val = df_1212class_val[['DX4_pred_1ST','DX4_pred_2ND','DX4_pred_3RD','DX4_pred_4TH']]
df_3111 = df_3111[['DX4_pred_1ST','DX4_pred_2ND','DX4_pred_3RD','DX4_pred_4TH']]
df_3111_val = df_3111_val[['DX4_pred_1ST','DX4_pred_2ND','DX4_pred_3RD','DX4_pred_4TH']]

####  此模型輸入為四個小模型的輸出，[年齡, 性別, 6CDRclass, CN_MCI_other, 1212class, 3111class]  請按照順序，共19個feature
####  此模型輸出為真實CDRSB值的分類，根據值分為六類
x_train = np.concatenate((df.iloc[:, [4,5,17,18,19,20,21,22]].values, df_class, df_1212class, df_3111),axis =1 ) 
y_train = df.iloc[:, [29]]   # [真實CDRSB值分類]
x_test = np.concatenate((df_val.iloc[:, [4,5,17,18,19,20,21,22]].values,df_class_val, df_1212class_val, df_3111_val),axis = 1)   
y_test = df_val.iloc[:, [29]]   # [真實CDRSB值分類]
x_train[:,0] = (x_train[:,0]-50)/10   # 年齡正則化
x_test[:,0] = (x_test[:,0]-50)/10    # 年齡正則化

In [ ]:
#### sample weights ####
count_y = []
count_y_train = df.iloc[:,[11,12,13,14,15,16]]
for i in range(len(count_y_train)):
    if (count_y_train.loc[i] == [1,0,0,0,0,0]).all():
        count_y.append(1)
    elif (count_y_train.loc[i] ==[0,1,0,0,0,0]).all():
        count_y.append(2)
    elif (count_y_train.loc[i] == [0,0,1,0,0,0]).all():
        count_y.append(3)
    elif (count_y_train.loc[i] == [0,0,0,1,0,0]).all():
        count_y.append(4)
    elif (count_y_train.loc[i] == [0,0,0,0,1,0]).all():
        count_y.append(5)
    elif (count_y_train.loc[i] == [0,0,0,0,0,1]).all():
        count_y.append(6)
        
a = count_y.count(1)
b = count_y.count(2)
c = count_y.count(3)
d = count_y.count(4)
e = count_y.count(5)
f = count_y.count(6)
all_count =a+b+c+d+e+f
sw_dic = [(all_count/a) , (all_count/b), (all_count/c), (all_count/d), (all_count/e), (all_count/f)]

sw = sw_dic[0]*np.ones(len(y_train))
for ii in range(len(y_train)):
    if (y_train.loc[ii] == 1 ).all():
        sw[ii] = sw_dic[0]
    elif (y_train.loc[ii] == 2 ).all():
        sw[ii] = sw_dic[1]
    elif( y_train.loc[ii] == 3 ).all():
        sw[ii] = sw_dic[2]
    elif (y_train.loc[ii] == 4 ).all():
        sw[ii] = sw_dic[3]
    elif (y_train.loc[ii] == 5 ).all():
        sw[ii] = sw_dic[4]
    elif (y_train.loc[ii] == 6 ).all():
        sw[ii] = sw_dic[5]


lm = LinearRegression()
lm.fit(x_train,y_train,sw)
predicted = lm.predict(x_test)
predicted = pd.DataFrame(predicted)

In [ ]:
df_da_true = []    # true class
for ii in range(len(y_test)):
    if (y_test.loc[ii] == 1).all():
        df_da_true.append('Normal')
    elif (y_test.loc[ii] == 2).all():
        df_da_true.append('Questionable impairment')
    elif(y_test.loc[ii] == 3).all():
        df_da_true.append('Very mild dementia')
    elif (y_test.loc[ii] == 4).all():
        df_da_true.append('Mild dementia')
    elif (y_test.loc[ii] == 5).all():
        df_da_true.append('Moderate dementia')
    elif (y_test.loc[ii] == 6).all():
        df_da_true.append('Severe dementia')

df = pd.DataFrame(predicted)
df_da = []    # pred class
for ii in range(len(df)):
    if (df.loc[ii] <= 1.66).all():
        df_da.append('Normal')
    elif ((df.loc[ii] > 1.66)&(df.loc[ii] <= 2.62)).all():
        df_da.append('Questionable impairment')
    elif ((df.loc[ii] > 2.62)&(df.loc[ii] <= 3.19)).all():
        df_da.append('Very mild dementia')
    elif ((df.loc[ii] > 3.19)&(df.loc[ii] <= 3.76)).all():
        df_da.append('Mild dementia')
    elif ((df.loc[ii] > 3.76)&(df.loc[ii] <= 4.71)).all():
        df_da.append('Moderate dementia')
    elif (df.loc[ii] > 4.71).all():
        df_da.append('Severe dementia')

df_da_xxx = []
for ii in range(len(df)):
    if (df.loc[ii] <= 1.66).all():
        df_da_xxx.append(1)
    elif ((df.loc[ii] > 1.66)&(df.loc[ii] <= 2.62)).all():
        df_da_xxx.append(2)
    elif ((df.loc[ii] > 2.62)&(df.loc[ii] <= 3.19)).all():
        df_da_xxx.append(3)
    elif ((df.loc[ii] > 3.19)&(df.loc[ii] <= 3.76)).all():
        df_da_xxx.append(4)
    elif ((df.loc[ii] > 3.76)&(df.loc[ii] <= 4.71)).all():
        df_da_xxx.append(5)
    elif (df.loc[ii] > 4.71).all():
        df_da_xxx.append(6)

In [ ]:
df_val, df_da_true, df_da, df_class_val = pd.DataFrame(df_val), pd.DataFrame(df_da_true), pd.DataFrame(df_da), pd.DataFrame(df_class_val)
df_da_1st = df_da.copy()


for j in range(len(df_class_val)):
    if (df_class_val.loc[j][0] > df_class_val.loc[j][1]) & (df_class_val.loc[j][0] > df_class_val.loc[j][2]):
        df_da_1st.loc[j][0]='Normal'

confusion = pd.DataFrame(np.concatenate((df_val, df_da_true,df_da,df_da_1st),axis =1))
confusion.rename(columns = {0:'PTID',1:'EXAMDATE',2:'VISCODE',3:'FOLDER',4:'TRUE_AGE',5:'PTGENDER',
                            6:'APOE4',7:'CDRSB',8:'CDRGL',9:'DX',10:'RACE',29:'true_CDR_class_value',
                            30:'true_CDR_class',31:'pred_CDR_class',32:'pred_CDR_class_1st'},inplace = True)

In [ ]:
# confusion.to_csv('fusion.csv')
from sklearn.metrics import confusion_matrix
label = ['Normal','Questionable impairment','Very mild dementia','Mild dementia','Moderate dementia','Severe dementia']  #类别集合
conf_mat = confusion_matrix(confusion.true_CDR_class, confusion.pred_CDR_class_1st, labels = label )
conf_mat = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]

df_cm = pd.DataFrame(conf_mat, index=label, columns=label)
plt.figure(figsize=(10,8))

heatmap = sns.heatmap(conf_mat,annot=True, fmt='0.2f', cmap='Blues', xticklabels = label,yticklabels = label)
heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right')
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right')

plt.ylabel("class_true label")
plt.xlabel("class_predict label")
plt.title("confusion matrix")
plt.show()